In [11]:
import pandas as pd

file_path = '../08. Evaluasi Pola (Pattern Evaluation)/Dataset/persib.csv'

data = pd.read_csv(file_path)

data

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1801647536496857484,Fri Jun 14 21:55:26 +0000 2024,29,@Indostransfer Asing Madura musim ini kemungki...,1801735224654762024,NaN,Indostransfer,in,NaN,0,5,0,https://x.com/wayuuu18/status/1801735224654762024,1387311427958886400,wayuuu18
1,1801647536496857484,Fri Jun 14 21:52:02 +0000 2024,0,@Indostransfer Sebelumnya *cmiiw jika Persib l...,1801734368849002701,NaN,Indostransfer,in,NaN,0,0,0,https://x.com/wayuuu18/status/1801734368849002701,1387311427958886400,wayuuu18
2,1801596984186302759,Fri Jun 14 21:50:38 +0000 2024,0,@FootyRankings Jeonbuk Port Lee man Persib,1801734016007344428,NaN,FootyRankings,in,Bandung,0,0,0,https://x.com/vrijeman___/status/1801734016007...,1180727063264686080,vrijeman___
3,1801647536496857484,Fri Jun 14 21:48:47 +0000 2024,0,@Indostransfer @vikyfauzy Disini yg main bang ...,1801733549768511897,NaN,Indostransfer,in,NaN,0,3,0,https://x.com/hayesstyls/status/18017335497685...,1799981034425208832,hayesstyls
4,1801663965564489825,Fri Jun 14 21:43:06 +0000 2024,0,@swilkinsonbc Anjir naon warna biruuu persib m...,1801732118323605980,NaN,swilkinsonbc,in,NaN,0,0,0,https://x.com/Labeaufleur/status/1801732118323...,1544115184343531520,Labeaufleur
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5054,1798896044400595095,Fri Jun 07 01:53:33 +0000 2024,0,https://t.co/xBDsqdmcEI,1798896044400595095,https://pbs.twimg.com/media/GPb2a2nbYAA3Bcs.jpg,NaN,zxx,NaN,0,0,0,https://x.com/anindyasinjai/status/17988960444...,2258214942,anindyasinjai
5055,1798896014994419875,Fri Jun 07 01:53:26 +0000 2024,0,https://t.co/6AK5lTgWMG,1798896014994419875,https://pbs.twimg.com/media/GPb2ZHUaEAAdogy.jpg,NaN,zxx,NaN,0,0,0,https://x.com/anindyasinjai/status/17988960149...,2258214942,anindyasinjai
5056,1798895997164351928,Fri Jun 07 01:53:22 +0000 2024,0,https://t.co/Bewddzz4Af,1798895997164351928,https://pbs.twimg.com/media/GPb2YCHaQAADJw1.jpg,NaN,zxx,NaN,0,0,0,https://x.com/anindyasinjai/status/17988959971...,2258214942,anindyasinjai
5057,1798895948934082931,Fri Jun 07 01:53:10 +0000 2024,0,https://t.co/5WfO1dUzbw,1798895948934082931,https://pbs.twimg.com/media/GPb2VLvaoAAKs-X.jpg,NaN,zxx,NaN,0,0,0,https://x.com/anindyasinjai/status/17988959489...,2258214942,anindyasinjai


In [12]:
# Import libraries
import string
import pandas as pd
import numpy as np
import nltk
import re
from nltk.tokenize import word_tokenize
from mpstemmer import MPStemmer
from nltk.corpus import stopwords
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

# Download necessary NLTK data
nltk.download('stopwords')

# Initialize stemmer
stemmer = MPStemmer()

# Load the model
model = load_model('../07. Pemodelan Data (Data Mining)/model/model-bilstm.h5')

class Emotion:
    @staticmethod
    def classify_emotion(data):
        def lower_case(text):
            return text.lower()

        def remove_tweet_special(text):
            text = text.replace('\\t', " ").replace('\\n', " ").replace('\\u', " ").replace('\\', "")
            text = text.encode('ascii', 'replace').decode('ascii')
            text = ' '.join(re.sub(r"([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)", " ", text).split())
            return text.replace("http://", " ").replace("https://", " ")
        
        # fill na data kosong
        def remove_empty_data(text):
            if text == ' ' or text == '  ' or text == '   ':
                return None
            return text
        

        def remove_number(text):
            return re.sub(r"\d+", "", text)

        def remove_punctuation(text):
            return text.translate(str.maketrans("", "", string.punctuation))

        def remove_whitespace_LT(text):
            return text.strip()

        def remove_whitespace_multiple(text):
            return re.sub(r'\s+', ' ', text)

        def remove_singl_char(text):
            return re.sub(r"\b[a-zA-Z]\b", "", text)

        def remove_repeated_char(text):
            return re.sub(r'(.)\1+', r'\1', text)

        def word_tokenize_wrapper(text):
            return word_tokenize(text)

        normalizad_word = pd.read_csv("./utils/kamus-alay.csv")
        normalizad_word_dict = {row[0]: row[1] for index, row in normalizad_word.iterrows()}

        def normalized_term(document):
            return [normalizad_word_dict.get(term, term) for term in document]

        def stem_wrapper(term):
            return [stemmer.stem(word) for word in term]

        stop_words = stopwords.words('indonesian')
        stop_words = [word for word in stop_words if word not in ['tidak', 'baik', 'jelek', 'jangan', 'belum', 'bukan', "enggak", "engga", "bener", "benar"]]
        stop_words.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 'kalo', 'amp', 'biar', 'bikin', 'bilang', 'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 'jd', 'jgn', 'sdh', 'aja', 'n', 't', 'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt', '&amp', 'yah'])

        txt_stopword = pd.read_csv("utils/stopwords.txt", names=["stopwords"], header=None)
        stop_words.extend(txt_stopword["stopwords"][0].split(' '))
        stop_words = set(stop_words)

        def stopwords_removal(words):
            return [word for word in words if word not in stop_words]

        def replace_nan_with_none(data):
            return data.applymap(lambda x: None if pd.isna(x) else x)

        df = pd.DataFrame(data)

        if 'predicted_label' not in df.columns:
            df['predicted_label'] = np.nan
            df['probability_emotion'] = np.nan

        to_process_df = df[df['predicted_label'].isna()]

        if not to_process_df.empty:
            to_process_df['processed_text'] = to_process_df['full_text'].apply(lower_case)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(remove_tweet_special)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(remove_empty_data)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(remove_number)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(remove_punctuation)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(remove_whitespace_LT)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(remove_whitespace_multiple)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(remove_singl_char)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(remove_repeated_char)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(word_tokenize_wrapper)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(normalized_term)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(stem_wrapper)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(stopwords_removal)
            to_process_df['processed_text'] = to_process_df['processed_text'].apply(' '.join)

            print("Text preprocessing done!")

            with open('../07. Pemodelan Data (Data Mining)/tokenizer-emotion.pickle', 'rb') as handle:
                tokenizer = pickle.load(handle)

            sequences = tokenizer.texts_to_sequences(to_process_df['processed_text'])
            padded_sequences = pad_sequences(sequences, maxlen=50, truncating='post', padding='post')

            predictions = model.predict(padded_sequences)
            
            emotion_labels = ['Neutral', 'Anger', 'Joy', 'Love', 'Sad', 'Fear']
            predicted_labels = []
            predicted_probabilities = []

            for pred in predictions:
                max_idx = np.argmax(pred)
                predicted_labels.append(emotion_labels[max_idx])
                predicted_probabilities.append({emotion_labels[i]: pred[i] for i in range(len(emotion_labels))})

            print("Prediction done!")

            to_process_df['predicted_label'] = predicted_labels
            to_process_df['probability_emotion'] = [predicted_probabilities[i][predicted_labels[i]] for i in range(len(predicted_labels))]

            df.update(to_process_df)

        df = replace_nan_with_none(df)

        return df
    
    @staticmethod
    def calculate_emotion_percentages(data):
        total = len(data)
        emotion_counts = {'Neutral': 0, 'Anger': 1, 'Joy': 2, 'Love': 3, 'Sad': 4, 'Fear': 5}

        for item in data:
            emotion_counts[item['predicted_label']] += 1
        
        percentages = {emotion: (count / total) * 100 for emotion, count in emotion_counts.items()}
        return percentages


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TEMP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
# Convert float values to strings
data['full_text'] = data['full_text'].astype(str)

# Call the classify_emotion method
hasil = Emotion.classify_emotion(data)

# Menyimpan hasil hanya full_text, prediksi dan probabilitas 
hasil = hasil[['username','full_text','predicted_label', 'probability_emotion']]
hasil.to_csv('hasil/hasil_prediksi-persib(bilstm).csv', index=False)

C:\Users\TEMP\AppData\Local\Temp\ipykernel_5668\2180982358.py:64: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  normalizad_word_dict = {row[0]: row[1] for index, row in normalizad_word.iterrows()}


Text preprocessing done!
159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
Prediction done!


C:\Users\TEMP\AppData\Local\Temp\ipykernel_5668\2180982358.py:134: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Anger' 'Neutral' 'Neutral' ... 'Love' 'Love' 'Love']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.update(to_process_df)
C:\Users\TEMP\AppData\Local\Temp\ipykernel_5668\2180982358.py:84: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return data.applymap(lambda x: None if pd.isna(x) else x)
